<a href="https://colab.research.google.com/github/jjevans25/English_to_Dutch_Transformer/blob/master/Transformer_EN_DUTCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import re
import time
from google.colab import drive

In [0]:
try:
  %tensorflow_version 2.x
except:
  pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [0]:
# Loading Files

In [0]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open("/content/drive/My Drive/NLP_Projects/Dutch_Transformer/europarl-v7.nl-en.en",
          mode='r',
          encoding="utf-8") as f:
      europarl_en = f.read()
with open("/content/drive/My Drive/NLP_Projects/Dutch_Transformer/europarl-v7.nl-en.nl",
          mode='r',
          encoding="utf-8") as f:
      europarl_nl = f.read()
with open("/content/drive/My Drive/NLP_Projects/Dutch_Transformer/nonbreaking_prefix.en",
          mode='r',
          encoding="utf-8") as f:
      non_breaking_prefix_en = f.read()
with open("/content/drive/My Drive/NLP_Projects/Dutch_Transformer/nonbreaking_prefix2.nl",
          mode='r',
          encoding="utf-8") as f:
      non_breaking_prefix_nl = f.read()

In [0]:
non_breaking_prefix_nl[:500]

'a\nb\nc\nd\ne\nf\ng\nh\ni\nj\nk\nl\nm\nn\no\np\nq\nr\ns\nt\nu\nv\nw\nx\ny\nz\nbacc\nbc\nbgen\nc.i\ndhr\ndr\ndr.h.c\ndrs\ndrs\nds\neint\nfa\nFa\nfam\ngen\ngenm\ning\nir\njhr\njkvr\njr\nkand\nkol\nlgen\nlkol\nLt\nmaj\nMej\nmevr\nMme\nmr\nmr\nMw\no.b.s\nplv\nprof\nritm\ntint\nVz\nZ.D\nZ.D.H\nZ.E\nZ.Em\nZ.H\nZ.K.H\nZ.K.M\nZ.M\nz.v\na.g.v\nbijv\nbijz\nbv\nd.w.z\ne.c\ne.g\ne.k\nev\ni.p.v\ni.s.m\ni.t.t\ni.v.m\nm.a.w\nm.b.t\nm.b.v\nm.h.o\nm.i\nm.i.v\nv.w.t\nNr \nNrs\nnrs\nnr'

In [0]:
# cleaning data

In [0]:
non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
non_breaking_prefix_en = [' ' + pref + '.' for pref in non_breaking_prefix_en]
non_breaking_prefix_nl = non_breaking_prefix_nl.split("\n")
non_breaking_prefix_nl = [' ' + pref + '.' for pref in non_breaking_prefix_nl]

In [0]:
corpus_en = europarl_en
# Add $$$ after non ending sentence points
for prefix in non_breaking_prefix_en:
    corpus_en = corpus_en.replace(prefix, prefix + '$$$')
corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_en)
# Remove $$$ markers
corpus_en = re.sub(r".\$\$\$", '', corpus_en)
# Clear multiple spaces
corpus_en = re.sub(r"  +", " ", corpus_en)
corpus_en = corpus_en.split('\n')

corpus_nl = europarl_nl
for prefix in non_breaking_prefix_nl:
    corpus_nl = corpus_nl.replace(prefix, prefix + '$$$')
corpus_nl = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_nl)
corpus_nl = re.sub(r".\$\$\$", '', corpus_nl)
corpus_nl = re.sub(r"  +", " ", corpus_nl)
corpus_nl = corpus_nl.split('\n')

In [0]:
# Tokenizing

In [0]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_en, target_vocab_size=2**13
)
tokenizer_nl = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_nl, target_vocab_size=2**13
)

In [0]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_NL = tokenizer_nl.vocab_size + 2

In [0]:
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
            for sentence in corpus_en]
outputs = [[VOCAB_SIZE_NL-2] + tokenizer_nl.encode(sentence) + [VOCAB_SIZE_NL-1]
            for sentence in corpus_nl]

In [0]:
# Max Length

In [0]:
MAX_LENGTH = 20
idx_to_remove = [count for count, sent in enumerate(inputs)
                if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
  del inputs[idx]
  del outputs[idx]
idx_to_remove = [count for count, sent in enumerate(outputs)
                if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
  del inputs[idx]
  del outputs[idx]

In [0]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding="post",
                                                       maxlen=MAX_LENGTH)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                       value=0,
                                                       padding="post",
                                                       maxlen=MAX_LENGTH)

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
# Model Building 

In [0]:
# Positional Encoding

Positional encoding formulae:

$PE_{(pos,2i)} =\sin(pos/10000^{2i/dmodel})$

$PE_{(pos,2i+1)} =\cos(pos/10000^{2i/dmodel})$

In [0]:
class PositionalEncoding(layers.Layer):

  def __init__(self):
    super(PositionalEncoding, self).__init__()

  def get_angles(self, pos, i, d_model): # pos : (seq_length, 1) i: (1, d_model)
    angles = 1 / np.power(10000., (2*(i//2))/np.float32(d_model))
    return pos * angles # (seq_length, d_model)

  def call(self, inputs):
    seq_length = inputs.shape.as_list()[-2]
    d_model = inputs.shape.as_list()[-1]
    angles = self.get_angles(np.arange(seq_length)[:, np.newaxis],
                             np.arange(d_model)[np.newaxis, :],
                             d_model)
    angles[:, 0::2] = np.sin(angles[:, 0::2])
    angles[:, 1::2] = np.cos(angles[:, 1::2])
    pos_encoding = angles[np.newaxis, ...]
    return inputs + tf.cast(pos_encoding, tf.float32)

In [0]:
#Attention

$Attention(Q, K, V ) = \text{softmax}\left(\dfrac{QK^T}{\sqrt{d_k}}\right)V $

In [0]:
def scaled_dot_product_attention(queries, keys, values, mask):
  product = tf.matmul(queries, keys, transpose_b=True)

  keys_dim = tf.cast(tf.shape(keys)[-1], tf.float32)
  scaled_product = product / tf.math.sqrt(keys_dim)

  if mask is not None:
    scaled_product += (mask * -1e9)

  attention = tf.matmul(tf.nn.softmax(scaled_product, axis=-1), values)

  return attention

In [0]:
# Multihead attention

In [0]:
class MultiHeadAttention(layers.Layer):

  def __init__(self, nb_proj):
    super(MultiHeadAttention, self).__init__()
    self.nb_proj = nb_proj

  def build(self, input_shape):
    self.d_model = input_shape[-1]
    assert self.d_model % self.nb_proj == 0

    self.d_proj = self.d_model // self.nb_proj

    self.query_lin = layers.Dense(units=self.d_model)
    self.key_lin = layers.Dense(units=self.d_model)
    self.value_lin = layers.Dense(units=self.d_model)

    self.final_lin = layers.Dense(units=self.d_model)

  def split_proj(self, inputs, batch_size): # inputs: (batch_size, seq_length, d_model)
    shape = (batch_size,
             -1,
             self.nb_proj,
             self.d_proj)
    splitted_inputs = tf.reshape(inputs, shape=shape) # (batch_size, nb_proj, seq_length, d_proj)
    return tf.transpose(splitted_inputs, perm=[0, 2, 1, 3]) # (batch_size, seq_length, nb_proj, d_proj)

  def call(self, queries, keys, values, mask):
    batch_size = tf.shape(queries)[0]

    queries = self.query_lin(queries)
    keys = self.key_lin(keys)
    values = self.value_lin(values)

    queries = self.split_proj(queries, batch_size)
    keys = self.split_proj(keys, batch_size)                          
    values = self.split_proj(values, batch_size)             

    attention = scaled_dot_product_attention(queries, keys, values, mask)    

    attention = tf.transpose(attention, perm=[0, 2, 1, 3])       

    concat_attention = tf.reshape(attention, 
                                  shape=(batch_size, -1, self.d_model))  
    
    outputs = self.final_lin(concat_attention)

    return outputs

In [0]:
# Encoder

In [0]:
class EncoderLayer(layers.Layer):

  def __init__(self, FFN_units, nb_proj, dropout_rate):
    super(EncoderLayer, self).__init__()
    self.FFN_units = FFN_units
    self.nb_proj = nb_proj
    self.dropout_rate = dropout_rate

  def build(self, input_shape):
    self.d_model = input_shape[-1]

    self.multi_head_attention = MultiHeadAttention(self.nb_proj)
    self.dropout_1 = layers.Dropout(rate=self.dropout_rate)
    self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

    self.dense_1 = layers.Dense(units=self.FFN_units, activation="relu")
    self.dense_2 = layers.Dense(units=self.d_model)
    self.dropout_2 = layers.Dropout(rate=self.dropout_rate)
    self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

  def call(self, inputs, mask, training):
    attention = self.multi_head_attention(inputs,
                                          inputs,
                                          inputs,
                                          mask)
    attention = self.dropout_1(attention, training=training)
    attention = self.norm_1(attention + inputs)

    outputs = self.dense_1(attention)
    outputs = self.dense_2(outputs)
    outputs = self.dropout_2(outputs, training=training)
    outputs = self.norm_2(outputs + attention)

    return outputs

In [0]:
class Encoder(layers.Layer):

  def __init__(self,
               nb_layers,
               FFN_units,
               nb_proj,
               dropout_rate,
               vocab_size,
               d_model,
               name="encoder"):
    super(Encoder, self).__init__(name=name)
    self.nb_layers = nb_layers
    self.d_model = d_model

    self.embedding = layers.Embedding(vocab_size, d_model)
    self.pos_encoding = PositionalEncoding()
    self.dropout = layers.Dropout(rate=dropout_rate)
    self.enc_layers = [EncoderLayer(FFN_units,
                                     nb_proj,
                                     dropout_rate)
                      for _ in range(nb_layers)]

  def call(self, inputs, mask, training):
    outputs = self.embedding(inputs)
    outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    outputs = self.pos_encoding(outputs)
    outputs = self.dropout(outputs, training)

    for i in range(self.nb_layers):
      outputs = self.enc_layers[i](outputs, mask, training)
    
    return outputs

In [0]:
# Decoder

In [0]:
class DecoderLayer(layers.Layer):

  def __init__(self, FFN_units, nb_proj, dropout_rate):
    super(DecoderLayer, self).__init__()
    self.FFN_units = FFN_units
    self.nb_proj = nb_proj
    self.dropout_rate = dropout_rate

  def build(self, input_shape):
    self.d_model = input_shape[-1]

    # self multi head attention
    self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
    self.dropout_1 = layers.Dropout(rate=self.dropout_rate)
    self.norm_1 = layers.LayerNormalization(epsilon=1e-6)

    # Multi head attention combined with encoder output
    self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
    self.dropout_2 = layers.Dropout(rate=self.dropout_rate)
    self.norm_2 = layers.LayerNormalization(epsilon=1e-6)

    # Feed forward
    self.dense_1 = layers.Dense(units=self.FFN_units,
                                activation="relu")
    self.dense_2 = layers.Dense(units=self.d_model)
    self.dropout_3 = layers.Dropout(rate=self.dropout_rate)
    self.norm_3 = layers.LayerNormalization(epsilon=1e-6)

  def call(self, inputs, enc_outputs, mask_1, mask_2, training):
    attention = self.multi_head_attention_1(inputs,
                                            inputs,
                                            inputs,
                                            mask_1)
    attention = self.dropout_1(attention, training)
    attention = self.norm_1(attention + inputs)

    attention_2 = self.multi_head_attention_2(attention,
                                              enc_outputs,
                                              enc_outputs,
                                              mask_2)
    
    attention_2 = self.dropout_2(attention_2, training)
    attention_2 = self.norm_2(attention_2 + attention)

    outputs = self.dense_1(attention_2)
    outputs = self.dense_2(outputs)
    outputs = self.dropout_3(outputs, training)
    outputs = self.norm_3(outputs + attention_2)

    return outputs

In [0]:
class Decoder(layers.Layer):

  def __init__(self,
               nb_layers,
               FFN_units,
               nb_proj,
               dropout_rate,
               vocab_size,
               d_model,
               name="decoder"):
    super(Decoder, self).__init__(name=name)
    self.d_model = d_model
    self.nb_layers = nb_layers

    self.embedding = layers.Embedding(vocab_size, d_model)
    self.pos_encoding = PositionalEncoding()
    self.dropout = layers.Dropout(rate=dropout_rate)
    
    self.dec_layers = [DecoderLayer(FFN_units,
                                    nb_proj,
                                    dropout_rate)
                      for _ in range(nb_layers)]

  def call(self, inputs, enc_outputs, mask_1, mask_2, training):
    outputs = self.embedding(inputs)
    outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    outputs = self.pos_encoding(outputs)
    outputs= self.dropout(outputs, training)

    for i in range(self.nb_layers):
      outputs = self.dec_layers[i](outputs,
                                   enc_outputs,
                                   mask_1,
                                   mask_2,
                                   training)
    return outputs 

In [0]:
# Transformer

In [0]:
class Transformer(tf.keras.Model):
    
    def __init__(self,
                 vocab_size_enc,
                 vocab_size_dec,
                 d_model,
                 nb_layers,
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 name="transformer"):
        super(Transformer, self).__init__(name=name)
        
        self.encoder = Encoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout_rate,
                               vocab_size_enc,
                               d_model)
        self.decoder = Decoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout_rate,
                               vocab_size_dec,
                               d_model)
        self.last_linear = layers.Dense(units=vocab_size_dec, name="lin_ouput")
    
    def create_padding_mask(self, seq):
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32)
        return mask[:, tf.newaxis, tf.newaxis, :]

    def create_look_ahead_mask(self, seq):
        seq_len = tf.shape(seq)[1]
        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        return look_ahead_mask
    
    def call(self, enc_inputs, dec_inputs, training):
        enc_mask = self.create_padding_mask(enc_inputs)
        dec_mask_1 = tf.maximum(
            self.create_padding_mask(dec_inputs),
            self.create_look_ahead_mask(dec_inputs)
        )
        dec_mask_2 = self.create_padding_mask(enc_inputs)
        
        enc_outputs = self.encoder(enc_inputs, enc_mask, training)
        dec_outputs = self.decoder(dec_inputs,
                                   enc_outputs,
                                   dec_mask_1,
                                   dec_mask_2,
                                   training)
        
        outputs = self.last_linear(dec_outputs)
        
        return outputs

In [0]:
# Training

In [0]:
tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 128 # 512
NB_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
NB_PROJ = 8 # 8
DROPOUT_RATE = 0.1 # 0.1

transformer = Transformer(vocab_size_enc=VOCAB_SIZE_EN,
                          vocab_size_dec=VOCAB_SIZE_NL,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout_rate=DROPOUT_RATE)

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction="none")

def loss_function(target, pred):
  mask = tf.math.logical_not(tf.math.equal(target, 0))
  loss_ = loss_object(target, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

leaning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(leaning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)

In [0]:
checkpoint_path = "./drive/My Drive/NLP Projects/Dutch_Transformer/ckpt/"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer
                           )

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("Latest checkpoint restored !!")

Latest checkpoint restored !!


In [0]:
EPOCHS = 10
for epoch in range(EPOCHS):
    print("Start of epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for (batch, (enc_inputs, targets)) in enumerate(dataset):
        dec_inputs = targets[:, :-1]
        dec_outputs_real = targets[:, 1:]
        with tf.GradientTape() as tape:
            predictions = transformer(enc_inputs, dec_inputs, True)
            loss = loss_function(dec_outputs_real, predictions)
        
        gradients = tape.gradient(loss, transformer.trainable_variables)
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
        
        train_loss(loss)
        train_accuracy(dec_outputs_real, predictions)
        
        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()))
            
    ckpt_save_path = ckpt_manager.save()
    print("Saving checkpoint for epoch {} at {}".format(epoch+1,
                                                        ckpt_save_path))
    print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

Start of epoch 1
Epoch 1 Batch 0 Loss 6.2617 Accuracy 0.0033
Epoch 1 Batch 50 Loss 6.1803 Accuracy 0.0167
Epoch 1 Batch 100 Loss 6.0741 Accuracy 0.0343
Epoch 1 Batch 150 Loss 6.0003 Accuracy 0.0404
Epoch 1 Batch 200 Loss 5.9101 Accuracy 0.0435
Epoch 1 Batch 250 Loss 5.8204 Accuracy 0.0453
Epoch 1 Batch 300 Loss 5.7001 Accuracy 0.0497
Epoch 1 Batch 350 Loss 5.5875 Accuracy 0.0560
Epoch 1 Batch 400 Loss 5.4742 Accuracy 0.0610
Epoch 1 Batch 450 Loss 5.3629 Accuracy 0.0651
Epoch 1 Batch 500 Loss 5.2622 Accuracy 0.0701
Epoch 1 Batch 550 Loss 5.1699 Accuracy 0.0748
Epoch 1 Batch 600 Loss 5.0807 Accuracy 0.0791
Epoch 1 Batch 650 Loss 4.9942 Accuracy 0.0831
Epoch 1 Batch 700 Loss 4.9180 Accuracy 0.0870
Epoch 1 Batch 750 Loss 4.8423 Accuracy 0.0907
Epoch 1 Batch 800 Loss 4.7753 Accuracy 0.0945
Epoch 1 Batch 850 Loss 4.7110 Accuracy 0.0983
Epoch 1 Batch 900 Loss 4.6508 Accuracy 0.1019
Epoch 1 Batch 950 Loss 4.5943 Accuracy 0.1054
Epoch 1 Batch 1000 Loss 4.5375 Accuracy 0.1088
Epoch 1 Batch 1050 

In [0]:
# Saving Model

In [0]:
# Evaluating

In [0]:
def evaluate(inp_sentence):
    inp_sentence = \
        [VOCAB_SIZE_EN-2] + tokenizer_en.encode(inp_sentence) + [VOCAB_SIZE_EN-1]
    enc_input = tf.expand_dims(inp_sentence, axis=0)
    
    output = tf.expand_dims([VOCAB_SIZE_NL-2], axis=0)
    
    for _ in range(MAX_LENGTH):
        predictions = transformer(enc_input, output, False)
        
        prediction = predictions[:, -1:, :]
        
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)
        
        if predicted_id == VOCAB_SIZE_NL-1:
            return tf.squeeze(output, axis=0)
        
        output = tf.concat([output, predicted_id], axis=-1)
        
    return tf.squeeze(output, axis=0)

In [0]:
def translate(sentence):
  output = evaluate(sentence).numpy()

  predicted_sentence = tokenizer_nl.decode(
      [i for i in output if i < VOCAB_SIZE_NL-2]
    )
  
  print("Input: {}".format(sentence))
  print("Predicted translation: {}".format(predicted_sentence))

In [0]:
translate("Today is a good day.")

NameError: ignored